<a href="https://colab.research.google.com/github/Aakashaakubhardwaj/Retail-Analysis-Sales-Forecasting/blob/main/Benz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing lybraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder , MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')

loading train dataset as df1

In [2]:
df1 = pd.read_csv('train.csv')

In [3]:
df1.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df1.shape

(4209, 378)

checking duplicate values, null values and separating numerical and categorical columns

In [5]:
df1.duplicated().sum()

0

In [6]:
df1.isnull().sum().sum()

0

In [7]:
train = df1.drop(columns= ['ID'],axis =1)

In [8]:
train_data = train.drop("y", axis = 1)

In [9]:
train_data.dtypes

X0      object
X1      object
X2      object
X3      object
X4      object
         ...  
X380     int64
X382     int64
X383     int64
X384     int64
X385     int64
Length: 376, dtype: object

In [10]:
train_cat = train_data.select_dtypes(include = object)
train_num = train_data.select_dtypes(exclude=object)

In [11]:
train_cat.columns

Index(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'], dtype='object')

In [12]:
train_cat.head()

,X0,X1,X2,X3,X4,X5,X6,X8
0,k,v,at,a,d,u,j,o
1,k,t,av,e,d,y,l,o
2,az,w,n,c,d,x,j,x
3,az,t,n,f,d,x,l,e
4,az,v,n,f,d,h,d,n


In [13]:
train_num.head()

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


checking the unique values in train dataset

In [14]:
for i in ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']:
    print()
    print('train_cat')
    print(i, train_cat[i].unique())



train_cat
X0 ['k' 'az' 't' 'al' 'o' 'w' 'j' 'h' 's' 'n' 'ay' 'f' 'x' 'y' 'aj' 'ak' 'am'
 'z' 'q' 'at' 'ap' 'v' 'af' 'a' 'e' 'ai' 'd' 'aq' 'c' 'aa' 'ba' 'as' 'i'
 'r' 'b' 'ax' 'bc' 'u' 'ad' 'au' 'm' 'l' 'aw' 'ao' 'ac' 'g' 'ab']

train_cat
X1 ['v' 't' 'w' 'b' 'r' 'l' 's' 'aa' 'c' 'a' 'e' 'h' 'z' 'j' 'o' 'u' 'p' 'n'
 'i' 'y' 'd' 'f' 'm' 'k' 'g' 'q' 'ab']

train_cat
X2 ['at' 'av' 'n' 'e' 'as' 'aq' 'r' 'ai' 'ak' 'm' 'a' 'k' 'ae' 's' 'f' 'd'
 'ag' 'ay' 'ac' 'ap' 'g' 'i' 'aw' 'y' 'b' 'ao' 'al' 'h' 'x' 'au' 't' 'an'
 'z' 'ah' 'p' 'am' 'j' 'q' 'af' 'l' 'aa' 'c' 'o' 'ar']

train_cat
X3 ['a' 'e' 'c' 'f' 'd' 'b' 'g']

train_cat
X4 ['d' 'b' 'c' 'a']

train_cat
X5 ['u' 'y' 'x' 'h' 'g' 'f' 'j' 'i' 'd' 'c' 'af' 'ag' 'ab' 'ac' 'ad' 'ae'
 'ah' 'l' 'k' 'n' 'm' 'p' 'q' 's' 'r' 'v' 'w' 'o' 'aa']

train_cat
X6 ['j' 'l' 'd' 'h' 'i' 'a' 'g' 'c' 'k' 'e' 'f' 'b']

train_cat
X8 ['o' 'x' 'e' 'n' 's' 'a' 'h' 'p' 'm' 'k' 'd' 'i' 'v' 'j' 'b' 'q' 'w' 'g'
 'y' 'l' 'f' 'u' 'r' 't' 'c']


finding the variance of train dataset and removing the columns having zero variance

In [15]:
train_var = train_num.var()
train_var

X10     0.013131
X11     0.000000
X12     0.069457
X13     0.054623
X14     0.244893
          ...   
X380    0.008015
X382    0.007547
X383    0.001661
X384    0.000475
X385    0.001424
Length: 368, dtype: float64

In [16]:
variable_var_zero = [ ]
columns = train_num.columns
for i in range(0,len(train_var)):
    if train_var[i]==0:
        variable_var_zero.append(columns[i])

In [17]:
np.ravel(variable_var_zero)

array(['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290',
       'X293', 'X297', 'X330', 'X347'], dtype='<U4')

In [18]:
train_new = train_num.drop(['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290',
       'X293', 'X297', 'X330', 'X347'],axis =1)
train_new.shape

(4209, 356)

In [19]:
train_new.head()

,X10,X12,X13,X14,X15,X16,X17,X18,X19,X20,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


applying label encoder to the categorical columns

In [20]:
le = LabelEncoder()
for i in train_cat.columns:
  train_cat[i] = le.fit_transform(train_cat[i])
train_cat.head()

,X0,X1,X2,X3,X4,X5,X6,X8
0,32,23,17,0,3,24,9,14
1,32,21,19,4,3,28,11,14
2,20,24,34,2,3,27,9,23
3,20,21,34,5,3,27,11,4
4,20,23,34,5,3,12,3,13


creating the final dataset by concatinating encoded columns and numerical columns

In [21]:
train_final = pd.concat([train_cat,train_new],axis =1)
train_final.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X12,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,32,23,17,0,3,24,9,14,0,0,...,0,0,1,0,0,0,0,0,0,0
1,32,21,19,4,3,28,11,14,0,0,...,1,0,0,0,0,0,0,0,0,0
2,20,24,34,2,3,27,9,23,0,0,...,0,0,0,0,0,0,1,0,0,0
3,20,21,34,5,3,27,11,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20,23,34,5,3,12,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0


repeating the same process to test dataset

In [22]:
df2 = pd.read_csv("test.csv")

In [23]:
df2.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [24]:
df2.shape

(4209, 377)

In [25]:
df2.duplicated().sum()

0

In [26]:
df2.isnull().sum().sum()

0

In [27]:
test = df2.drop(columns= ['ID'],axis =1)

In [28]:
test_cat = test.select_dtypes(include = object)
test_num = test.select_dtypes(exclude=object)

In [29]:
for i in ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']:
    print()
    print('test_cat')
    print(i, test_cat[i].unique())


test_cat
X0 ['az' 't' 'w' 'y' 'x' 'f' 'ap' 'o' 'ay' 'al' 'h' 'z' 'aj' 'd' 'v' 'ak'
 'ba' 'n' 'j' 's' 'af' 'ax' 'at' 'aq' 'av' 'm' 'k' 'a' 'e' 'ai' 'i' 'ag'
 'b' 'am' 'aw' 'as' 'r' 'ao' 'u' 'l' 'c' 'ad' 'au' 'bc' 'g' 'an' 'ae' 'p'
 'bb']

test_cat
X1 ['v' 'b' 'l' 's' 'aa' 'r' 'a' 'i' 'p' 'c' 'o' 'm' 'z' 'e' 'h' 'w' 'g' 'k'
 'y' 't' 'u' 'd' 'j' 'q' 'n' 'f' 'ab']

test_cat
X2 ['n' 'ai' 'as' 'ae' 's' 'b' 'e' 'ak' 'm' 'a' 'aq' 'ag' 'r' 'k' 'aj' 'ay'
 'ao' 'an' 'ac' 'af' 'ax' 'h' 'i' 'f' 'ap' 'p' 'au' 't' 'z' 'y' 'aw' 'd'
 'at' 'g' 'am' 'j' 'x' 'ab' 'w' 'q' 'ah' 'ad' 'al' 'av' 'u']

test_cat
X3 ['f' 'a' 'c' 'e' 'd' 'g' 'b']

test_cat
X4 ['d' 'b' 'a' 'c']

test_cat
X5 ['t' 'b' 'a' 'z' 'y' 'x' 'h' 'g' 'f' 'j' 'i' 'd' 'c' 'af' 'ag' 'ab' 'ac'
 'ad' 'ae' 'ah' 'l' 'k' 'n' 'm' 'p' 'q' 's' 'r' 'v' 'w' 'o' 'aa']

test_cat
X6 ['a' 'g' 'j' 'l' 'i' 'd' 'f' 'h' 'c' 'k' 'e' 'b']

test_cat
X8 ['w' 'y' 'j' 'n' 'm' 's' 'a' 'v' 'r' 'o' 't' 'h' 'c' 'k' 'p' 'u' 'd' 'g'
 'b' 'q' 'e' 'l' 'f' 'i' 'x']


In [30]:
test_var = test_num.var()
test_var

X10     0.018650
X11     0.000238
X12     0.068851
X13     0.057345
X14     0.244859
          ...   
X380    0.008015
X382    0.008715
X383    0.000475
X384    0.000712
X385    0.001661
Length: 368, dtype: float64

In [31]:
var_test_zero = [ ]
columns = test_num.columns
for i in range(0,len(test_var)):
  if test_var[i]==0:
    var_test_zero.append(columns[i])

In [32]:
np.ravel(var_test_zero)

array(['X257', 'X258', 'X295', 'X296', 'X369'], dtype='<U4')

In [33]:
test_new = test_num.drop(['X257', 'X258', 'X295', 'X296', 'X369'],axis =1)
test_new.shape

(4209, 363)

In [34]:
for i in train_cat.columns:
  test_cat[i] = le.fit_transform(test_cat[i])
test_cat.head()

,X0,X1,X2,X3,X4,X5,X6,X8
0,21,23,34,5,3,26,0,22
1,42,3,8,0,3,9,6,24
2,21,23,17,5,3,0,9,9
3,21,13,34,5,3,31,11,13
4,45,20,17,2,3,30,8,12


In [35]:
test_final = pd.concat([test_cat,test_new],axis =1)
test_final.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,21,23,34,5,3,26,0,22,0,0,...,0,0,0,1,0,0,0,0,0,0
1,42,3,8,0,3,9,6,24,0,0,...,0,0,1,0,0,0,0,0,0,0
2,21,23,17,5,3,0,9,9,0,0,...,0,0,0,1,0,0,0,0,0,0
3,21,13,34,5,3,31,11,13,0,0,...,0,0,0,1,0,0,0,0,0,0
4,45,20,17,2,3,30,8,12,0,0,...,1,0,0,0,0,0,0,0,0,0


In [36]:
test_final.shape

(4209, 371)

reducing the dimentionality of both datasets using PCA

In [37]:
pca = PCA(n_components = 0.98,random_state=42)

In [38]:
X = train_final
Y = train.y

x_test = test_final

In [39]:
X_train,X_val,Y_train,Y_val = train_test_split(X,Y,test_size = 0.2,random_state = 42)

In [40]:
X.shape

(4209, 364)

In [41]:
Y.shape

(4209,)

In [42]:
n_comp = 12
pca = PCA(n_components=n_comp, random_state=420)

In [43]:
pca_X_train =pd.DataFrame(pca.fit_transform(X_train))
pca_X_val = pd.DataFrame(pca.fit_transform(X_val))
pca_test = pd.DataFrame(pca.fit_transform(x_test))

In [44]:
pca.explained_variance_ratio_

array([0.43515102, 0.17670897, 0.13646292, 0.10977912, 0.08622208,
       0.01433962, 0.00722966, 0.00406843, 0.00274007, 0.00228112,
       0.00221525, 0.00182721])

Now predicting the test_df using Xgboost

In [45]:
model = xgb.XGBRegressor(objective='reg:linear',learning_rate=0.1)

In [46]:
model.fit(pca_X_train,Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, objective='reg:linear', ...)

In [51]:
y_pred = model.predict(pca_X_val)
acc_score = model.score(pca_X_train,Y_train)
print(acc_score)

0.8064735824938614


In [54]:
p_test = model.predict(pca_test)
p_test

array([ 87.668945,  97.39477 ,  91.13108 , ...,  97.25337 , 116.12511 ,
        93.26952 ], dtype=float32)

In [56]:
Predicted_Data = pd.DataFrame()
Predicted_Data['y'] = p_test
Predicted_Data

,y
0,87.668945
1,97.394768
2,91.131081
3,84.744087
4,101.820534
...,...
4204,109.009781
4205,96.463669
4206,97.253372
4207,116.125107


Predicted values of "y" for test dataset